# Load necessary libraries

In [1]:
import torch
import numpy as np
import torch.nn as nn 
from torchsummary import summary
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from data import RPSDataset
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import random
from model_architecture import RPSNet

# Test if cuda is available

In [2]:
torch.cuda.is_available()

True

# Seed for reproducibility

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 42
set_seed(seed)

# Load Model

In [4]:

# Define the model architecture again (must match the saved model's architecture)
model = RPSNet(num_classes=4);

# Load the saved model state dictionary
model.load_state_dict(torch.load("trained_model_5.pth"))

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device);

# Set the model to evaluation mode
model.eval();

# Test model on dummy input

In [5]:
dummy_input = torch.randn(1, 3, 416, 416).to(device)
output = model(dummy_input)
print(output.shape)

torch.Size([1, 4])


# Summary

In [7]:
# Print model summary
summary(model, input_size=(3, 416, 416), device=str(device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 416, 416]             216
       BatchNorm2d-2          [-1, 8, 416, 416]              16
              Mish-3          [-1, 8, 416, 416]               0
         ConvBlock-4          [-1, 8, 416, 416]               0
            Conv2d-5          [-1, 8, 208, 208]             576
       BatchNorm2d-6          [-1, 8, 208, 208]              16
              Mish-7          [-1, 8, 208, 208]               0
         ConvBlock-8          [-1, 8, 208, 208]               0
            Conv2d-9          [-1, 2, 208, 208]             144
      BatchNorm2d-10          [-1, 2, 208, 208]               4
             Mish-11          [-1, 2, 208, 208]               0
        ConvBlock-12          [-1, 2, 208, 208]               0
           Conv2d-13          [-1, 2, 208, 208]             180
      BatchNorm2d-14          [-1, 2, 2

# Loading datasets

In [8]:
# Training dataset path
img_dir = "Dataset-5/train/images"
labels_dir = "Dataset-5/train/labels"

# Validation dataset path
validation_labels_dir = "Dataset-5/valid/labels"
validation_img_dir = "Dataset-5/valid/images"

batch_size = 8

# Create dataset and dataloader
dataset = RPSDataset(img_dir, labels_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create validation dataset and dataloader
validation_dataset = RPSDataset(validation_img_dir, validation_labels_dir)
validation_dataloader = DataLoader(validation_dataset, batch_size, shuffle=True)

# Display size of the batch sizes
for images, labels in dataloader:
    print("Images batch shape:", images.shape)
    print("Labels batch shape:", labels.shape)
    break

print()

for images, labels in validation_dataloader:
    print("Images validation batch shape:", images.shape)
    print("Labels validation batch shape:", labels.shape)
    break

Images batch shape: torch.Size([8, 3, 416, 416])
Labels batch shape: torch.Size([8, 4])

Images validation batch shape: torch.Size([8, 3, 416, 416])
Labels validation batch shape: torch.Size([8, 4])


# Train model

In [ ]:
def train(model, train_loader, val_loader, loss_function, optimizer, num_epochs):
    # Train on cuda if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
    
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_function(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            
            # Calculate training accuracy
            preds = torch.argmax(outputs, dim=1)
            true_labels = torch.argmax(labels, dim=1)
            correct_train += (preds == true_labels).sum().item()
            total_train += labels.size(0)

        # Get and display the loss and accuracy
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        train_accuracy = correct_train / total_train
        train_accuracies.append(train_accuracy)
        print(f"Epoch {epoch + 1} / {num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation loss and accuracy
        model.eval()
        val_running_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                val_outputs = model(val_images)
                val_loss = loss_function(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_images.size(0)
                
                # Calculate validation accuracy
                val_preds = torch.argmax(val_outputs, dim=1)
                val_true_labels = torch.argmax(val_labels, dim=1)
                correct_val += (val_preds == val_true_labels).sum().item()
                total_val += val_labels.size(0)

        # Display the validation loss and accuracy
        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_losses.append(val_epoch_loss)
        val_accuracy = correct_val / total_val
        val_accuracies.append(val_accuracy)
        print(f"Epoch {epoch + 1} / {num_epochs}, Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Save the model
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

    return train_losses, val_losses, train_accuracies, val_accuracies

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=3e-4)
loss_function = nn.BCEWithLogitsLoss()
    
# Train the model and save it
save_path = "RPSNet/trained_model_5.pth"
train_losses, val_losses, train_accuracies, val_accuracies = train(model, dataloader, validation_dataloader, loss_function, optimizer, num_epochs=60)

# Display metrics
Display the metrics of the model which is used in the report

In [ ]:

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Epochs')
    plt.legend()
    plt.show()

def plot_accuracies(train_accuracies, val_accuracies):
    plt.figure(figsize=(10, 5))
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy over Epochs')
    plt.legend()
    plt.show()

def plot_confusion_matrix(true_labels, predicted_labels, class_names):
    cm = confusion_matrix(true_labels, predicted_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

def evaluate_model(model, dataloader):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            true_labels = torch.argmax(labels, dim=1).cpu().numpy()
            
            all_preds.extend(preds)
            all_labels.extend(true_labels)
    
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return all_labels, all_preds

class_names = ['Paper', 'Rock', 'Scissor', 'None']

# Evaluate the model
true_labels, predicted_labels = evaluate_model(model, validation_dataloader)
    
# Visualize the loss
plot_losses(train_losses, val_losses)

# Visualize the accuracy
plot_accuracies(train_accuracies, val_accuracies)

# Plot the confusion matrix
plot_confusion_matrix(true_labels, predicted_labels, class_names)